In [ ]:
from IPython.display import display, Markdown
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

embedding = OllamaEmbeddings(model="qwen2.5")
llm = ChatOllama(
    model="qwen2.5",
    temperature=0.0,
    max_tokens=100,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.0,
)

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [3]:
# 加载向量库，其中包含了所有课程材料的 Embedding。
from langchain.vectorstores import Chroma
import panel as pn  # GUI

# pn.extension()

persist_directory = "../docs/chroma/matplotlib"
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

question = "这门课的主要内容是什么？"
docs = vectordb.similarity_search(question, k=3)
print(len(docs))

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_72815/1153810359.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


3


In [6]:
# pretty_print_docs(docs)

In [7]:
# 构建 prompt
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# 运行 chain
from langchain.chains import RetrievalQA

question = "这门课的主题是什么？"
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)


result = qa_chain({"query": question})
print(result["result"])

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_72815/2878909522.py:26: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


这门课的主题是关于使用matplotlib库进行数据可视化，涵盖了从基础概念到具体图表绘制的各种知识点。谢谢你的提问！


In [8]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True,  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_72815/771698300.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [16]:
from langchain.chains import ConversationalRetrievalChain

retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

question = "这门课会学习 Python 吗？"
result = qa({"question": question})
print(result["answer"])

这门课程主要介绍使用 Python 的 matplotlib 库进行数据可视化，但并没有直接提到是否教授 Python 语言的基础知识。因此，根据提供的上下文信息，我们无法确定这门课是否会学习 Python。不过通常情况下，学习使用 matplotlib 进行数据可视化的课程都会涵盖一定的 Python 基础知识。如果你对 Python 编程感兴趣或不熟悉，可能需要在开始之前先自学一些基础知识。


In [17]:
question = "那么我最好会python才能学习这门课程？"
result = qa({"question": question})
print(result["answer"])

虽然掌握Python对学习这门课程有帮助，但并不是唯一的要求。这段文档主要介绍了数据可视化的基本概念和绘图方法，并没有深入依赖复杂的编程知识。如果你对编程有一定了解，特别是Python的话，可以更容易理解和应用这些概念。但是，即使你不是特别熟悉Python，也可以通过逐步学习来掌握必要的技能。

重点在于理解图表的构建过程、各种绘图函数的作用以及如何根据需求调整和优化图表。因此，如果你愿意投入时间和精力去学习Python基础，并结合文档中的示例进行实践，那么你完全可以跟上课程的内容。


In [ ]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader

from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

import panel as pn  # GUI
import param


def load_db(file, chain_type, k):
    """
    该函数用于加载 PDF 文件，切分文档，生成文档的嵌入向量，创建向量数据库，定义检索器，并创建聊天机器人实例。

    参数:
    file (str): 要加载的 PDF 文件路径。
    chain_type (str): 链类型，用于指定聊天机器人的类型。
    k (int): 在检索过程中，返回最相似的 k 个结果。

    返回:
    qa (ConversationalRetrievalChain): 创建的聊天机器人实例。
    """
    # 载入文档
    loader = PyPDFLoader(file)
    documents = loader.load()
    # 切分文档
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # 定义 Embeddings
    embeddings = OllamaEmbeddings(model="qwen2.5")
    # 根据数据创建向量数据库
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # 定义检索器
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # 创建 chatbot 链，Memory 由外部管理
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOllama(model="qwen2.5", temperature=0.0, max_tokens=100, top_p=0.9, top_k=50, repetition_penalty=1.0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
        # memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
        
    )
    return qa


# 定义文件输入小部件
file_input = pn.widgets.FileInput()
# 定义加载按钮
button_load = pn.widgets.Button(name="Load File", button_type="primary")
# 定义文本输入小部件
inp = pn.widgets.TextInput(name='Input', placeholder='Enter text here...')

# 用于存储聊天记录、回答、数据库查询和回复
class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query = param.String("")
    db_response = param.List([])

    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []
        self.loaded_file = "../docs/matplotlib/第一回：Matplotlib初相识.pdf"
        self.qa = load_db(self.loaded_file, "stuff", 4)

    # 将文档加载到聊天机器人中
    def call_load_db(self, count):
        """
        count: 数量
        """
        if count == 0 or file_input.value is None:  # 初始化或未指定文件 :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # 本地副本
            self.loaded_file = file_input.filename
            button_load.button_style = "outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style = "solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    # 处理对话链
    def convchain(self, query):
        """
        query: 用户的查询
        """
        if not query:
            return pn.WidgetBox(
                pn.Row("User:", pn.pane.Markdown("", width=600)), scroll=True
            )
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result["answer"]
        self.panels.extend(
            [
                pn.Row("User:", pn.pane.Markdown(query, width=600)),
                pn.Row(
                    "ChatBot:",
                    pn.pane.Markdown(
                        self.answer, width=600, style={"background-color": "#F6F6F6"}
                    ),
                ),
            ]
        )
        inp.value = ""  # 清除时清除装载指示器
        return pn.WidgetBox(*self.panels, scroll=True)

    # 获取最后发送到数据库的问题
    @param.depends(
        "db_query ",
    )
    def get_lquest(self):
        if not self.db_query:
            return pn.Column(
                pn.Row(
                    pn.pane.Markdown(
                        f"Last question to DB:", styles={"background-color": "#F6F6F6"}
                    )
                ),
                pn.Row(pn.pane.Str("no DB accesses so far")),
            )
        return pn.Column(
            pn.Row(
                pn.pane.Markdown(f"DB query:", styles={"background-color": "#F6F6F6"})
            ),
            pn.pane.Str(self.db_query),
        )

    # 获取数据库返回的源文件
    @param.depends(
        "db_response",
    )
    def get_sources(self):
        if not self.db_response:
            return
        rlist = [
            pn.Row(
                pn.pane.Markdown(
                    f"Result of DB lookup:", styles={"background-color": "#F6F6F6"}
                )
            )
        ]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    # 获取当前聊天记录
    @param.depends("convchain", "clr_history")
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(
                pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True
            )
        rlist = [
            pn.Row(
                pn.pane.Markdown(
                    f"Current Chat History variable",
                    styles={"background-color": "#F6F6F6"},
                )
            )
        ]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    # 清除聊天记录
    def clr_history(self, count=0):
        self.chat_history = []
        return

In [ ]:
# 初始化聊天机器人
cb = cbfs()

# 定义界面的小部件
file_input = pn.widgets.FileInput(accept=".pdf")  # PDF 文件的文件输入小部件
button_load = pn.widgets.Button(
    name="Load DB", button_type="primary"
)  # 加载数据库的按钮
button_clearhistory = pn.widgets.Button(
    name="Clear History", button_type="warning"
)  # 清除聊天记录的按钮
button_clearhistory.on_click(cb.clr_history)  # 将清除历史记录功能绑定到按钮上
inp = pn.widgets.TextInput(
    placeholder="Enter text here…"
)  # 用于用户查询的文本输入小部件

# 将加载数据库和对话的函数绑定到相应的部件上
bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.Image("./img/convchain.jpg")

# 使用 Panel 定义界面布局
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2 = pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources),
)
tab3 = pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4 = pn.Column(
    pn.Row(file_input, button_load, bound_button_load),
    pn.Row(
        button_clearhistory,
        pn.pane.Markdown("Clears chat history. Can use to start a new topic"),
    ),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400)),
)
# 将所有选项卡合并为一个仪表盘
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown("# ChatWithYourData_Bot")),
    pn.Tabs(
        ("Conversation", tab1),
        ("Database", tab2),
        ("Chat History", tab3),
        ("Configure", tab4),
    ),
)
# pn.extension()
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'2da7c27b-ef1b-4cbf-a5b7-8b74dbe572f0': {'version…

In [ ]:
from datetime import datetime

current_time = datetime.now().strftime("%H:%M:%S")
print(f"Current time: {current_time}")